In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.dynamical_systems import dare, moas_closed_loop
from pympc.control import MPCHybridController
from pympc.feasible_set_library import FeasibleSetLibrary, load_library
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt

# Construction of Box-Atlas

Robot moving limbs

In [3]:
from pympc.models.boxatlas_parameters import weight, stiffness
penetration = weight / stiffness

# right hand
A_rh = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[1., 0.],[-1., 0.],[0., -1.]]),
}
b_rh = {
    'not_in_contact': np.array([[.4],[.5]]),
    'in_contact': np.array([[-.4],[.4+penetration],[.5]]),
}
contact_surfaces_rh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rh = np.array([[-.3], [.0]])
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh)

# left foot
A_lf = {
    'not_in_contact': np.array([[0., -1.]]),
    'in_contact': np.array([[0., 1.],[0., -1.]]),
}
b_lf = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+2.*penetration]]),
}
contact_surfaces_lf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lf = np.array([[.2], [-.5 - penetration / 2.]])
lf = MovingLimb(A_lf, b_lf, contact_surfaces_lf, q_lf)

# right foot
A_rf = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[0., 1.],[-1., 0.],[0., -1.]]),
}
b_rf = {
    'not_in_contact': np.array([[.5],[.5]]),
    'in_contact': np.array([[-.5],[.5],[.5+2.*penetration]]),
}
contact_surfaces_rf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rf = np.array([[-.2], [-.5 - penetration / 2.]])
rf = MovingLimb(A_rf, b_rf, contact_surfaces_rf, q_rf)

Assemble robot

In [4]:
limbs = { 'moving': {'rh': rh, 'lf': lf, 'rf': rf}, 'fixed': {} }
nominal_mode = {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
box_atlas = BoxAtlas(limbs, nominal_mode)

Print state, input, and modes (a mode indicates in which domain each moving limb is, domains are numbered as they are listed in the limb list)

In [5]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
print 'Box-atlas modes:\n', box_atlas.contact_modes

Box-Atlas states:
['qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
Box-Atlas inputs:
['vlfx', 'vlfy', 'vrfx', 'vrfy', 'vrhx', 'vrhy']
Box-atlas modes:
[{'lf': 'not_in_contact', 'rf': 'not_in_contact', 'rh': 'not_in_contact'}, {'lf': 'not_in_contact', 'rf': 'not_in_contact', 'rh': 'in_contact'}, {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}, {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}, {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'not_in_contact'}, {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'in_contact'}, {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}, {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}]


# Hybrid MPC Controller

In [6]:
from pympc.geometry.polytope import LowerDimensionalPolytope
from pympc.control import canonical_reachability_decomposition

crd = canonical_reachability_decomposition(
    box_atlas.nominal_system.A,
    box_atlas.nominal_system.B
    )
n_R, T, T_R, T_N, A_RR, B_R = [crd['n_R'], crd['T'], crd['T_R'], crd['T_N'], crd['A_RR'], crd['B_R']]
Q_R = T_R.T.dot(box_atlas.Q).dot(T_R)
P_R, K_R = dare(A_RR, B_R, Q_R, box_atlas.R)
#print np.linalg.eig(A_RR + B_R.dot(K_R))[0]
D_lhs_x = box_atlas.nominal_domain.lhs_min[:,:box_atlas.n_x]
D_lhs_u = box_atlas.nominal_domain.lhs_min[:,box_atlas.n_x:]
D_lhs_R = np.hstack((D_lhs_x.dot(T_R), D_lhs_u))
D_R = Polytope(
    D_lhs_R,
    box_atlas.nominal_domain.rhs_min
    )
D_R.assemble()
X_N_R = moas_closed_loop(A_RR, B_R, K_R, D_R)
T_inv = np.linalg.inv(T)
T_inv_R = T_inv[:n_R,:]
T_inv_N = T_inv[n_R:,:]
X_N = LowerDimensionalPolytope(
    X_N_R.lhs_min.dot(T_inv_R),
    X_N_R.rhs_min,
    T_inv_N,
    np.zeros((T_inv_N.shape[0], 1))
    )
P = T_inv_R.T.dot(P_R).dot(T_inv_R)
K = K_R.dot(T_inv_R)

Computation of Maximal Invariant Constraint-Admissible Set started.
Determinedness index: 6, Convergence index: -0.000598162798726, Number of facets: 210.    
Maximal Invariant Constraint-Admissible Set found.
Removing redundant facets ... minimal facets are 71.


Hybrid MPC controller

In [7]:
N = 10
objective_norm = 'two'
controller = MPCHybridController(box_atlas.pwa_system, N, objective_norm, box_atlas.Q, box_atlas.R, P, X_N)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter MIPGap to 1e-06
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001


Trick to avoid forbidden transitions

In [ ]:
controller = box_atlas.avoid_forbidden_transitions(controller)

## Visualizer

In [ ]:
# ['qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = .1
x_0[2,0] = .1
u, x, ss = controller.feedforward(x_0)[0:3]

Optimize a model with 6885 rows, 1232 columns and 34021 nonzeros
Model has 297 quadratic objective terms
Model has 10 SOS constraints
Variable types: 1152 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [7e-05, 3e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-19, 2e+00]
Presolve removed 2929 rows and 118 columns
Presolve time: 0.10s
Presolved: 3956 rows, 1114 columns, 24991 nonzeros
Presolved model has 274 quadratic objective terms
Variable types: 1042 continuous, 72 integer (72 binary)

Root relaxation: objective 3.963733e-02, 7044 iterations, 0.34 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.03964    0   23          -    0.03964      -     -    0s
H    0     0                       9.9369285    0.03964   100%     -    0s
H    0     0         

In [ ]:
from pympc.models.boxatlas_parameters import sampling_time

box_atlas.visualize(x[0])
time.sleep(20 * sampling_time)
for k in range(N):
    box_atlas.visualize(x[k])
    time.sleep(sampling_time)

In [ ]:
print ss
box_atlas.print_mode_sequence(ss)

# ['qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_test = np.array([
    [0.], # 'qlfx'
    [0.], # 'qlfy'
    [0.], # 'qrfx'
    [0.], # 'qrfy'
    [0.], # 'qrhx'
    [0.], # 'qrhy'
    [0.], # 'qbx'
    [0.], # 'qby'
    [0.], # 'tb'
    [0.], # 'vbx'
    [0.], # 'vby'
    [0.] # 'ob'
])
box_atlas.visualize(x_test)